In [ ]:
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 31.6 MB/s 
     |████████████████████████████████| 26.3 MB 46.1 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
!pip install sewar

  Created wheel for sewar: filename=sewar-0.4.4-py3-none-any.whl size=10390 sha256=5c70fa645f169fd94c75513ec95f4a0b63db6860a5f4060b765d5056904c9601
  Stored in directory: /root/.cache/pip/wheels/b5/00/df/f1ba40d0d44c95b0db52804e518c128d5631866b91ea1960ba
Successfully built sewar


In [ ]:
from art.estimators.classification.blackbox import BlackBoxClassifierNeuralNetwork, BlackBoxClassifier
import art.utils
import numpy as np
from art.defences.preprocessor import FeatureSqueezing, JpegCompression, SpatialSmoothing
import tensorflow as tf
#tf.enable_eager_execution()
tf.compat.v1.disable_eager_execution()
import tensorflow_hub as hub
import numpy as np
import matplotlib 
#mpl.use('tkagg')
import matplotlib.pyplot as plt
%matplotlib inline 
#import tensorflow_text
import os
from art.estimators.classification import KerasClassifier
import art.utils
# from art.utils import load_dataset
from art.attacks.extraction import *
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, DeepFool, CarliniL2Method,BrendelBethgeAttack
from art.attacks.evasion import BoundaryAttack
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

In [ ]:
!pip install -U 'fsspec>=0.3.3'

     |████████████████████████████████| 132 kB 5.0 MB/s 


In [ ]:
import tracemalloc
import pandas as pd
import dask.dataframe as dd
import time

In [ ]:
def tracing_start():
    tracemalloc.stop()
    print("nTracing Status : ", tracemalloc.is_tracing())
    tracemalloc.start()
    print("Tracing Status : ", tracemalloc.is_tracing())
def tracing_mem():
    first_size, first_peak = tracemalloc.get_traced_memory()
    peak = first_peak/(1024*1024)
    print("Peak Size in MB - ", peak)

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = art.utils.load_dataset(str("cifar10")) 
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size))
print("y_train shape: " + str(y_train.shape) + "\n" + "x_train size: " + str(y_train.size))
print("x_test shape: " + str(x_test.shape) + "\n" + "x_train size: " + str(x_test.size))
print("y_test shape: " + str(y_test.shape) + "\n" + "x_train size: " + str(y_test.size))


# make a subset of test data
idx = np.arange(len(x_test))
subset_x_test = []
subset_y_test = []
y_label = np.where(y_test > 0)[1]
for i in range(10):
    mask = y_label == i
    subset_y_test.append(y_test[mask][10:20])
    subset_x_test.append(x_test[mask][10:20])

subset_x_test = np.concatenate(subset_x_test,axis=0)
subset_y_test = np.concatenate(subset_y_test,axis=0)

x_train shape: (50000, 32, 32, 3)
x_train size: 153600000
y_train shape: (50000, 10)
x_train size: 500000
x_test shape: (10000, 32, 32, 3)
x_train size: 30720000
y_test shape: (10000, 10)
x_train size: 100000


In [ ]:
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
model = hub.KerasLayer(model_url, input_shape=(32, 32, 3))
victim_model = tf.keras.Sequential([
        model,
        tf.keras.layers.Activation('softmax')
])

victim_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
victim_model.summary()
victim_classifier = KerasClassifier(model=victim_model, clip_values=(0., 1.))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 10)                7796426   
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                                 
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


#PoisoningAttackAdversarialEmbedding

In [ ]:
from art.attacks.poisoning.backdoor_attack import PoisoningAttackBackdoor
from art.attacks.poisoning.adversarial_embedding_attack import PoisoningAttackAdversarialEmbedding
from art.attacks.poisoning.perturbations import add_pattern_bd

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D
from tensorflow.keras.models import Model
from art.estimators.classification.keras import KerasClassifier

In [ ]:
subset_x_train_1 = x_train[0:500]
subset_y_train_1 = y_train[0:500]

subset_x_train_2 = x_train[0:1000]
subset_y_train_2 = y_train[0:1000]

subset_x_train_3 = x_train[0:2000]
subset_y_train_3 = y_train[0:2000]

In [ ]:
target = np.ones(10)
target_idx = 9
target[target_idx] = 2

tracing_start()
start = time.time()

backdoor = PoisoningAttackBackdoor(add_pattern_bd)
emb_attack = PoisoningAttackAdversarialEmbedding(victim_classifier, backdoor, 1, target,clone=False)
classifier = emb_attack.poison_estimator(subset_x_train_1, subset_y_train_1, nb_epochs=1)

end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

nTracing Status :  False
Tracing Status :  True
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
Instructions for updating:
Colocations handled automatically by placer.


Train on 500 samples
500/500 [==============================] - 9s 17ms/sample - loss: -1.4483 - sequential_1_loss: 6.3968 - backdoor_detect_loss: 0.7767 - sequential_1_accuracy: 0.9580 - backdoor_detect_accuracy: 0.5020
time elapsed 11717.175960540771 milli seconds
Peak Size in MB -  20.40753173828125


In [ ]:
import sklearn

In [ ]:
# Evaluate the classifier on the adversarial examples
preds = np.argmax(classifier.predict(subset_x_test), axis=1)
acc = np.sum(preds == np.argmax(subset_y_test, axis=1)) / subset_y_test.shape[0]
print("\n Accuracy of classifier on PoisoningAttackAdversarialEmbedding 500 training: %.2f%%" % (acc * 100))

predict_maxvalue = preds
dependent_maxvalue = np.argmax(subset_y_test, axis=1)

print('accuracy',sklearn.metrics.accuracy_score(dependent_maxvalue,predict_maxvalue))
print('precision',sklearn.metrics.precision_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('recall',sklearn.metrics.recall_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('f1-score',sklearn.metrics.f1_score(dependent_maxvalue,predict_maxvalue, average='weighted'))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,



 Accuracy of classifier on PoisoningAttackAdversarialEmbedding 500 training: 97.00%
accuracy 0.97
precision 0.9697979797979798
recall 0.97
f1-score 0.9694486215538848


In [ ]:
target = np.ones(10)
target_idx = 9
target[target_idx] = 2

tracing_start()
start = time.time()

#backdoor = PoisoningAttackBackdoor(add_pattern_bd)
#emb_attack = PoisoningAttackAdversarialEmbedding(victim_classifier, backdoor, 1, target,clone=False)
classifier_2 = emb_attack.poison_estimator(subset_x_train_2, subset_y_train_2, nb_epochs=1)

end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

# Evaluate the classifier on the adversarial examples
preds = np.argmax(classifier_2.predict(subset_x_test), axis=1)
acc = np.sum(preds == np.argmax(subset_y_test, axis=1)) / subset_y_test.shape[0]
print("\n Accuracy of classifier on PoisoningAttackAdversarialEmbedding 1000 training: %.2f%%" % (acc * 100))

predict_maxvalue = preds
dependent_maxvalue = np.argmax(subset_y_test, axis=1)

print('accuracy',sklearn.metrics.accuracy_score(dependent_maxvalue,predict_maxvalue))
print('precision',sklearn.metrics.precision_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('recall',sklearn.metrics.recall_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('f1-score',sklearn.metrics.f1_score(dependent_maxvalue,predict_maxvalue, average='weighted'))

nTracing Status :  False
Tracing Status :  True
Train on 1000 samples
1000/1000 [==============================] - 0s 418us/sample - loss: -1.9687 - sequential_1_loss: 6.5014 - backdoor_detect_loss: 0.8563 - sequential_1_accuracy: 0.9520 - backdoor_detect_accuracy: 0.4770
time elapsed 604.4697761535645 milli seconds
Peak Size in MB -  29.195640563964844

 Accuracy of classifier on PoisoningAttackAdversarialEmbedding 1000 training: 97.00%
accuracy 0.97
precision 0.9697979797979798
recall 0.97
f1-score 0.9694486215538848


In [ ]:
target = np.ones(10)
target_idx = 9
target[target_idx] = 2

tracing_start()
start = time.time()

#backdoor = PoisoningAttackBackdoor(add_pattern_bd)
#emb_attack = PoisoningAttackAdversarialEmbedding(victim_classifier, backdoor, 1, target,clone=False)
classifier_3 = emb_attack.poison_estimator(subset_x_train_3, subset_y_train_3, nb_epochs=1)

end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

# Evaluate the classifier on the adversarial examples
preds = np.argmax(classifier_3.predict(subset_x_test), axis=1)
acc = np.sum(preds == np.argmax(subset_y_test, axis=1)) / subset_y_test.shape[0]
print("\n Accuracy of classifier on PoisoningAttackAdversarialEmbedding 2000 training: %.2f%%" % (acc * 100))

predict_maxvalue = preds
dependent_maxvalue = np.argmax(subset_y_test, axis=1)

print('accuracy',sklearn.metrics.accuracy_score(dependent_maxvalue,predict_maxvalue))
print('precision',sklearn.metrics.precision_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('recall',sklearn.metrics.recall_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('f1-score',sklearn.metrics.f1_score(dependent_maxvalue,predict_maxvalue, average='weighted'))

nTracing Status :  False
Tracing Status :  True
Train on 2000 samples
2000/2000 [==============================] - 1s 370us/sample - loss: -5.1198 - sequential_1_loss: 5.7473 - backdoor_detect_loss: 1.1054 - sequential_1_accuracy: 0.9570 - backdoor_detect_accuracy: 0.4615
time elapsed 1022.0663547515869 milli seconds
Peak Size in MB -  54.73549461364746

 Accuracy of classifier on PoisoningAttackAdversarialEmbedding 2000 training: 97.00%
accuracy 0.97
precision 0.9697979797979798
recall 0.97
f1-score 0.9694486215538848


#FeatureCollisionAttack

In [ ]:
from art.attacks.poisoning.feature_collision_attack import FeatureCollisionAttack

In [ ]:
def poison_dataset(classifier, x_clean, y_clean):
        x_poison = np.copy(x_clean)
        y_poison = np.copy(y_clean)
        base = np.expand_dims(x_clean[0], axis=0)
        target = np.expand_dims(x_clean[1], axis=0)
        feature_layer = classifier.layer_names[-1]
        attack = FeatureCollisionAttack(classifier, target, feature_layer, max_iter=1)
        attack, attack_label = attack.poison(base)
        x_poison = np.append(x_poison, attack, axis=0)
        y_poison = np.append(y_poison, attack_label, axis=0)

        return x_poison, y_poison

In [ ]:
x_adv, y_adv = poison_dataset(victim_classifier, x_train, y_train)

Feature collision:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tracing_start()
start = time.time()

model_1 = victim_classifier
x_adv_1, y_adv_1 = poison_dataset(victim_classifier, subset_x_train_1, subset_y_train_1)
model_1.fit(x_adv_1, y_adv_1, nb_epochs=1, batch_size=32)

end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

# Evaluate the classifier on the adversarial examples
preds = np.argmax(model_1.predict(subset_x_test), axis=1)
acc = np.sum(preds == np.argmax(subset_y_test, axis=1)) / subset_y_test.shape[0]
print("\n Accuracy of classifier on FeatureCollisionAttack 500 training: %.2f%%" % (acc * 100))

predict_maxvalue = preds
dependent_maxvalue = np.argmax(subset_y_test, axis=1)

print('accuracy',sklearn.metrics.accuracy_score(dependent_maxvalue,predict_maxvalue))
print('precision',sklearn.metrics.precision_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('recall',sklearn.metrics.recall_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('f1-score',sklearn.metrics.f1_score(dependent_maxvalue,predict_maxvalue, average='weighted'))

nTracing Status :  False
Tracing Status :  True


Feature collision:   0%|          | 0/1 [00:00<?, ?it/s]

Train on 501 samples
501/501 [==============================] - 1s 1ms/sample - loss: 0.0323 - accuracy: 0.9980
time elapsed 1225.6858348846436 milli seconds
Peak Size in MB -  35.41802978515625

 Accuracy of classifier on FeatureCollisionAttack 500 training: 97.00%
accuracy 0.97
precision 0.9697979797979798
recall 0.97
f1-score 0.9694486215538848


In [ ]:
tracing_start()
start = time.time()

model_2 = victim_classifier
x_adv_2, y_adv_2 = poison_dataset(victim_classifier, subset_x_train_2, subset_y_train_2)
model_2.fit(x_adv_2, y_adv_2, nb_epochs=1, batch_size=32)

end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

# Evaluate the classifier on the adversarial examples
preds = np.argmax(model_2.predict(subset_x_test), axis=1)
acc = np.sum(preds == np.argmax(subset_y_test, axis=1)) / subset_y_test.shape[0]
print("\n Accuracy of classifier on FeatureCollisionAttack 1000 training: %.2f%%" % (acc * 100))

predict_maxvalue = preds
dependent_maxvalue = np.argmax(subset_y_test, axis=1)

print('accuracy',sklearn.metrics.accuracy_score(dependent_maxvalue,predict_maxvalue))
print('precision',sklearn.metrics.precision_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('recall',sklearn.metrics.recall_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('f1-score',sklearn.metrics.f1_score(dependent_maxvalue,predict_maxvalue, average='weighted'))

nTracing Status :  False
Tracing Status :  True


Feature collision:   0%|          | 0/1 [00:00<?, ?it/s]

Train on 1001 samples
1001/1001 [==============================] - 0s 307us/sample - loss: 0.0172 - accuracy: 0.9990
time elapsed 566.3485527038574 milli seconds
Peak Size in MB -  70.5872573852539

 Accuracy of classifier on FeatureCollisionAttack 1000 training: 97.00%
accuracy 0.97
precision 0.9697979797979798
recall 0.97
f1-score 0.9694486215538848


In [ ]:
tracing_start()
start = time.time()

model_3 = victim_classifier
x_adv_3_fe, y_adv_3_fe = poison_dataset(victim_classifier, subset_x_test, subset_y_test)
model_1.fit(x_adv_3_fe, y_adv_3_fe, nb_epochs=1, batch_size=32)

end = time.time()
print("time elapsed {} milli seconds".format((end-start)*1000))
tracing_mem()

# Evaluate the classifier on the adversarial examples
preds = np.argmax(model_3.predict(subset_x_test), axis=1)
acc = np.sum(preds == np.argmax(subset_y_test, axis=1)) / subset_y_test.shape[0]
print("\n Accuracy of classifier on FeatureCollisionAttack 2000 training: %.2f%%" % (acc * 100))

predict_maxvalue = preds
dependent_maxvalue = np.argmax(subset_y_test, axis=1)

print('accuracy',sklearn.metrics.accuracy_score(dependent_maxvalue,predict_maxvalue))
print('precision',sklearn.metrics.precision_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('recall',sklearn.metrics.recall_score(dependent_maxvalue,predict_maxvalue, average='weighted'))
print('f1-score',sklearn.metrics.f1_score(dependent_maxvalue,predict_maxvalue, average='weighted'))

nTracing Status :  False
Tracing Status :  True


Feature collision:   0%|          | 0/1 [00:00<?, ?it/s]

Train on 101 samples
101/101 [==============================] - 0s 337us/sample - loss: 0.1574 - accuracy: 0.9604
time elapsed 272.247314453125 milli seconds
Peak Size in MB -  7.283014297485352

 Accuracy of classifier on FeatureCollisionAttack 2000 training: 97.00%
accuracy 0.97
precision 0.9697979797979798
recall 0.97
f1-score 0.9694486215538848
